In [26]:
# Celda 1: Importaciones de librerías de modelado y carga de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

# Librerías de Scikit-Learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelos a probar
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier

# Métricas de evaluación
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [27]:
# Celda 2 (Unificada y Definitiva): Carga de datos y definición de todas las variables

# Cargar el dataset COMPLETO de las 3 energías
df = pd.read_csv('../data/processed/dataset_modelo_energias_renovables.csv')
print("Dataset completo de energías renovables cargado.")
display(df.head())

# --- VARIABLES PREDICTORAS (X) ---
# Son las mismas para todos los modelos
features = ['provincia', 'altitud', 'temp_promedio_anual_C', 'viento_promedio_anual_ms', 'potencial_hidrico_proxy_mm']
X = df[features]

# --- VARIABLES OBJETIVO (y) PARA LA ENERGÍA SOLAR ---
y_regresion_solar = df['energia_solar_anual_kwh']
y_clasificacion_solar = df['viabilidad_solar']

# --- VARIABLES OBJETIVO (y) PARA LA ENERGÍA EÓLICA ---
y_regresion_eolica = df['energia_eolica_anual_kwh']
y_clasificacion_eolica = df['viabilidad_eolica']

# --- VARIABLES OBJETIVO (y) PARA LA ENERGÍA HÍDRICA ---
y_regresion_hidrica = df['potencial_hidrico_proxy_mm']
y_clasificacion_hidrica = df['viabilidad_hidrica']

# --- PREPROCESADOR ---
# Identificar columnas categóricas y numéricas
categorical_features = ['provincia']
numerical_features = ['altitud', 'temp_promedio_anual_C', 'viento_promedio_anual_ms', 'potencial_hidrico_proxy_mm']

# Crear el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

print("\n--- Features y targets para TODOS los modelos definidos correctamente ---")

Dataset completo de energías renovables cargado.


,provincia,año,altitud,temp_promedio_anual_C,viento_promedio_anual_ms,potencial_hidrico_proxy_mm,potencia_solar_media_anual_kw,energia_solar_anual_kwh,viabilidad_solar,potencia_eolica_media_anual_w,energia_eolica_anual_kwh,viabilidad_eolica,viabilidad_hidrica
0,Azuay,2019,2560,13.130822,3.323616,1077.24,1.979583,17341.146,Bajo,68434.334780,599484.772674,Alto,Medio
1,Azuay,2020,2560,13.589317,3.194399,860.99,2.068855,18172.824,Alto,59889.909196,526072.962378,Alto,Bajo
2,Azuay,2021,2560,13.016658,3.100274,1160.65,1.991193,17442.852,Bajo,54874.723459,480702.577504,Alto,Medio
3,Azuay,2022,2560,13.055562,3.035233,952.06,1.925845,16870.398,Bajo,52153.404235,456863.821097,Medio,Bajo
4,Azuay,2023,2560,13.499918,3.028822,1072.91,1.998297,17505.078,Bajo,51012.715932,446871.391565,Medio,Medio



--- Features y targets para TODOS los modelos definidos correctamente ---


In [38]:
# Celda 2.1: Validación Cruzada para una Comparación Robusta de Modelos

from sklearn.model_selection import KFold, cross_val_score

# Configurar la validación cruzada
# n_splits=10 significa 10-Fold Cross-Validation
# shuffle=True mezcla los datos antes de dividirlos, bueno para datasets sin orden temporal
# random_state=42 asegura reproducibilidad
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

print("--- Validación Cruzada para Modelos de REGRESIÓN (Energía Solar) ---")
modelos_regresion_cv = {
    "Regresión Lineal": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(random_state=42, n_estimators=100),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42, n_estimators=100)
}

resultados_cv_regresion = {}
for nombre, modelo in modelos_regresion_cv.items():
    # Crear el pipeline
    pipeline_cv = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('regressor', modelo)])
    
    # Realizar la validación cruzada.
    # scoring='neg_root_mean_squared_error' -> cross_val_score maximiza, así que usamos el negativo de RMSE.
    # Luego lo multiplicamos por -1 para obtener el RMSE positivo.
    cv_scores = cross_val_score(pipeline_cv, X, y_regresion_solar, cv=kfold, scoring='neg_root_mean_squared_error')
    
    # Guardar resultados (media y desviación estándar del RMSE)
    resultados_cv_regresion[nombre] = {
        'RMSE_CV_Mean': -cv_scores.mean(),  # Media del RMSE
        'RMSE_CV_Std': cv_scores.std()       # Desviación estándar del RMSE
    }
    print(f"Modelo: {nombre} -> RMSE Promedio (CV): {-cv_scores.mean():.2f} (+/- {cv_scores.std():.2f})")

print("\n--- Tabla Comparativa de Modelos de Regresión (con Cross-Validation) ---")
df_resultados_cv_reg = pd.DataFrame(resultados_cv_regresion).T
display(df_resultados_cv_reg.sort_values(by='RMSE_CV_Mean'))


print("\n--- Validación Cruzada para Modelos de CLASIFICACIÓN (Viabilidad Solar) ---")
# Codificar el target para clasificación
le_solar_cv = LabelEncoder()
y_clasificacion_solar_encoded = le_solar_cv.fit_transform(y_clasificacion_solar)

modelos_clasificacion_cv = {
    "Regresión Logística": LogisticRegression(random_state=42, max_iter=1000),
    "Random Forest Classifier": RandomForestClassifier(random_state=42, n_estimators=100),
    "Gradient Boosting Classifier": GradientBoostingClassifier(random_state=42, n_estimators=100)
}

resultados_cv_clasificacion = {}
for nombre, modelo in modelos_clasificacion_cv.items():
    pipeline_cv = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', modelo)])
    
    # scoring='accuracy' para clasificación
    cv_scores = cross_val_score(pipeline_cv, X, y_clasificacion_solar_encoded, cv=kfold, scoring='accuracy')
    
    resultados_cv_clasificacion[nombre] = {
        'Accuracy_CV_Mean': cv_scores.mean(),
        'Accuracy_CV_Std': cv_scores.std()
    }
    print(f"Modelo: {nombre} -> Accuracy Promedio (CV): {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

print("\n--- Tabla Comparativa de Modelos de Clasificación (con Cross-Validation) ---")
df_resultados_cv_clf = pd.DataFrame(resultados_cv_clasificacion).T
display(df_resultados_cv_clf.sort_values(by='Accuracy_CV_Mean', ascending=False))

--- Validación Cruzada para Modelos de REGRESIÓN (Energía Solar) ---
Modelo: Regresión Lineal -> RMSE Promedio (CV): 536.41 (+/- 86.25)
Modelo: Random Forest Regressor -> RMSE Promedio (CV): 784.75 (+/- 326.55)
Modelo: Gradient Boosting Regressor -> RMSE Promedio (CV): 714.03 (+/- 309.24)

--- Tabla Comparativa de Modelos de Regresión (con Cross-Validation) ---


,RMSE_CV_Mean,RMSE_CV_Std
Regresión Lineal,536.407532,86.249429
Gradient Boosting Regressor,714.025042,309.235289
Random Forest Regressor,784.745497,326.551986



--- Validación Cruzada para Modelos de CLASIFICACIÓN (Viabilidad Solar) ---
Modelo: Regresión Logística -> Accuracy Promedio (CV): 0.5250 (+/- 0.1710)
Modelo: Random Forest Classifier -> Accuracy Promedio (CV): 0.5667 (+/- 0.1965)
Modelo: Gradient Boosting Classifier -> Accuracy Promedio (CV): 0.5500 (+/- 0.1590)

--- Tabla Comparativa de Modelos de Clasificación (con Cross-Validation) ---


,Accuracy_CV_Mean,Accuracy_CV_Std
Random Forest Classifier,0.566667,0.196497
Gradient Boosting Classifier,0.550000,0.158990
Regresión Logística,0.525000,0.170986


In [28]:
# Celda 3: División Train/Test para Energía Solar

# División para la tarea de Regresión Solar
X_train_solar_reg, X_test_solar_reg, y_train_solar_reg, y_test_solar_reg = train_test_split(
    X, y_regresion_solar, test_size=0.2, random_state=42
)

# División para la tarea de Clasificación Solar
X_train_solar_clf, X_test_solar_clf, y_train_solar_clf, y_test_solar_clf = train_test_split(
    X, y_clasificacion_solar, test_size=0.2, random_state=42, stratify=y_clasificacion_solar
)

print("Divisiones Train/Test completadas para los modelos SOLARES.")

Divisiones Train/Test completadas para los modelos SOLARES.


In [29]:
# Celda 4: Entrenamiento y Evaluación de Modelos de Regresión Solar

modelos_regresion = {
    "Regresión Lineal": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(random_state=42, n_estimators=100),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42, n_estimators=100)
}

resultados_regresion_solar = {}

print("--- Entrenando y evaluando modelos de REGRESIÓN SOLAR ---\n")

for nombre, modelo in modelos_regresion.items():
    pipeline_reg = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('regressor', modelo)])
    pipeline_reg.fit(X_train_solar_reg, y_train_solar_reg)
    y_pred = pipeline_reg.predict(X_test_solar_reg)
    
    mae = mean_absolute_error(y_test_solar_reg, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_solar_reg, y_pred))
    r2 = r2_score(y_test_solar_reg, y_pred)
    
    resultados_regresion_solar[nombre] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
    
    print(f"--- Resultados para: {nombre} (Solar) ---")
    print(f"Error Absoluto Medio (MAE): {mae:.2f} kWh")
    print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f} kWh")
    print(f"Coeficiente de Determinación (R²): {r2:.4f}")
    print("-" * 35)

df_resultados_reg_solar = pd.DataFrame(resultados_regresion_solar).T
print("\n--- Tabla Comparativa de Modelos de Regresión Solar ---")
display(df_resultados_reg_solar.sort_values(by='RMSE'))

--- Entrenando y evaluando modelos de REGRESIÓN SOLAR ---

--- Resultados para: Regresión Lineal (Solar) ---
Error Absoluto Medio (MAE): 371.08 kWh
Raíz del Error Cuadrático Medio (RMSE): 478.93 kWh
Coeficiente de Determinación (R²): 0.9650
-----------------------------------
--- Resultados para: Random Forest Regressor (Solar) ---
Error Absoluto Medio (MAE): 531.34 kWh
Raíz del Error Cuadrático Medio (RMSE): 800.06 kWh
Coeficiente de Determinación (R²): 0.9022
-----------------------------------
--- Resultados para: Gradient Boosting Regressor (Solar) ---
Error Absoluto Medio (MAE): 563.10 kWh
Raíz del Error Cuadrático Medio (RMSE): 780.95 kWh
Coeficiente de Determinación (R²): 0.9068
-----------------------------------

--- Tabla Comparativa de Modelos de Regresión Solar ---


,MAE,RMSE,R2
Regresión Lineal,371.078745,478.928578,0.964951
Gradient Boosting Regressor,563.104343,780.951952,0.906807
Random Forest Regressor,531.335436,800.058193,0.902191


In [30]:
# Celda 5: Entrenamiento y Evaluación de Modelos de Clasificación Solar

# Codificar la variable objetivo 'viabilidad_solar' a números
le_solar = LabelEncoder()
y_train_solar_clf_encoded = le_solar.fit_transform(y_train_solar_clf)
y_test_solar_clf_encoded = le_solar.transform(y_test_solar_clf)

modelos_clasificacion = {
    "Regresión Logística": LogisticRegression(random_state=42, max_iter=1000),
    "Random Forest Classifier": RandomForestClassifier(random_state=42, n_estimators=100),
    "Gradient Boosting Classifier": GradientBoostingClassifier(random_state=42, n_estimators=100)
}

resultados_clasificacion_solar = {}

print("--- Entrenando y evaluando modelos de CLASIFICACIÓN SOLAR ---\n")

for nombre, modelo in modelos_clasificacion.items():
    pipeline_clf = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', modelo)])
    pipeline_clf.fit(X_train_solar_clf, y_train_solar_clf_encoded)
    y_pred_encoded = pipeline_clf.predict(X_test_solar_clf)
    
    accuracy = accuracy_score(y_test_solar_clf_encoded, y_pred_encoded)
    resultados_clasificacion_solar[nombre] = {'Accuracy': accuracy}
    
    print(f"--- Resultados para: {nombre} (Solar) ---")
    print(f"Exactitud (Accuracy): {accuracy:.4f}")
    print("\nReporte de Clasificación:")
    print(classification_report(y_test_solar_clf_encoded, y_pred_encoded, target_names=le_solar.classes_))
    print("-" * 35)

df_resultados_clf_solar = pd.DataFrame(resultados_clasificacion_solar).T
print("\n--- Tabla Comparativa de Modelos de Clasificación Solar ---")
display(df_resultados_clf_solar.sort_values(by='Accuracy', ascending=False))

--- Entrenando y evaluando modelos de CLASIFICACIÓN SOLAR ---

--- Resultados para: Regresión Logística (Solar) ---
Exactitud (Accuracy): 0.5417

Reporte de Clasificación:
              precision    recall  f1-score   support

        Alto       0.75      0.75      0.75         8
        Bajo       0.42      0.62      0.50         8
       Medio       0.50      0.25      0.33         8

    accuracy                           0.54        24
   macro avg       0.56      0.54      0.53        24
weighted avg       0.56      0.54      0.53        24

-----------------------------------
--- Resultados para: Random Forest Classifier (Solar) ---
Exactitud (Accuracy): 0.4167

Reporte de Clasificación:
              precision    recall  f1-score   support

        Alto       0.62      0.62      0.62         8
        Bajo       0.36      0.50      0.42         8
       Medio       0.20      0.12      0.15         8

    accuracy                           0.42        24
   macro avg       0.40  

,Accuracy
Regresión Logística,0.541667
Random Forest Classifier,0.416667
Gradient Boosting Classifier,0.416667


In [31]:
# Celda 6: División Train/Test para Energía Eólica

# División para la tarea de Regresión Eólica
X_train_eolica_reg, X_test_eolica_reg, y_train_eolica_reg, y_test_eolica_reg = train_test_split(
    X, y_regresion_eolica, test_size=0.2, random_state=42
)

# División para la tarea de Clasificación Eólica
X_train_eolica_clf, X_test_eolica_clf, y_train_eolica_clf, y_test_eolica_clf = train_test_split(
    X, y_clasificacion_eolica, test_size=0.2, random_state=42, stratify=y_clasificacion_eolica
)

print("Divisiones Train/Test completadas para los modelos EÓLICOS.")

Divisiones Train/Test completadas para los modelos EÓLICOS.


In [32]:
# Celda 7: Entrenamiento y Evaluación de Modelos de Regresión Eólica

# (Reutilizamos el diccionario de modelos y el bucle de la celda 4)
resultados_regresion_eolica = {}

print("--- Entrenando y evaluando modelos de REGRESIÓN EÓLICA ---\n")

for nombre, modelo in modelos_regresion.items():
    pipeline_reg = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('regressor', modelo)])
    pipeline_reg.fit(X_train_eolica_reg, y_train_eolica_reg)
    y_pred = pipeline_reg.predict(X_test_eolica_reg)
    
    mae = mean_absolute_error(y_test_eolica_reg, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_eolica_reg, y_pred))
    r2 = r2_score(y_test_eolica_reg, y_pred)
    
    resultados_regresion_eolica[nombre] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
    
    print(f"--- Resultados para: {nombre} (Eólica) ---")
    print(f"Error Absoluto Medio (MAE): {mae:.2f} kWh")
    print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f} kWh")
    print(f"Coeficiente de Determinación (R²): {r2:.4f}")
    print("-" * 35)

df_resultados_reg_eolica = pd.DataFrame(resultados_regresion_eolica).T
print("\n--- Tabla Comparativa de Modelos de Regresión Eólica ---")
display(df_resultados_reg_eolica.sort_values(by='RMSE'))

--- Entrenando y evaluando modelos de REGRESIÓN EÓLICA ---

--- Resultados para: Regresión Lineal (Eólica) ---
Error Absoluto Medio (MAE): 15162.31 kWh
Raíz del Error Cuadrático Medio (RMSE): 19645.05 kWh
Coeficiente de Determinación (R²): 0.9966
-----------------------------------
--- Resultados para: Random Forest Regressor (Eólica) ---
Error Absoluto Medio (MAE): 31318.33 kWh
Raíz del Error Cuadrático Medio (RMSE): 46520.90 kWh
Coeficiente de Determinación (R²): 0.9809
-----------------------------------
--- Resultados para: Gradient Boosting Regressor (Eólica) ---
Error Absoluto Medio (MAE): 28989.88 kWh
Raíz del Error Cuadrático Medio (RMSE): 45917.56 kWh
Coeficiente de Determinación (R²): 0.9814
-----------------------------------

--- Tabla Comparativa de Modelos de Regresión Eólica ---


,MAE,RMSE,R2
Regresión Lineal,15162.307872,19645.052932,0.996587
Gradient Boosting Regressor,28989.877048,45917.563053,0.981355
Random Forest Regressor,31318.330251,46520.896669,0.980862


In [33]:
# Celda 8: Entrenamiento y Evaluación de Modelos de Clasificación Eólica

# Codificar la variable objetivo 'viabilidad_eolica' a números
le_eolica = LabelEncoder()
y_train_eolica_clf_encoded = le_eolica.fit_transform(y_train_eolica_clf)
y_test_eolica_clf_encoded = le_eolica.transform(y_test_eolica_clf)

# (Reutilizamos el diccionario de modelos y el bucle de la celda 5)
resultados_clasificacion_eolica = {}

print("--- Entrenando y evaluando modelos de CLASIFICACIÓN EÓLICA ---\n")

for nombre, modelo in modelos_clasificacion.items():
    pipeline_clf = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', modelo)])
    pipeline_clf.fit(X_train_eolica_clf, y_train_eolica_clf_encoded)
    y_pred_encoded = pipeline_clf.predict(X_test_eolica_clf)
    
    accuracy = accuracy_score(y_test_eolica_clf_encoded, y_pred_encoded)
    resultados_clasificacion_eolica[nombre] = {'Accuracy': accuracy}
    
    print(f"--- Resultados para: {nombre} (Eólica) ---")
    print(f"Exactitud (Accuracy): {accuracy:.4f}")
    print("\nReporte de Clasificación:")
    print(classification_report(y_test_eolica_clf_encoded, y_pred_encoded, target_names=le_eolica.classes_))
    print("-" * 35)

df_resultados_clf_eolica = pd.DataFrame(resultados_clasificacion_eolica).T
print("\n--- Tabla Comparativa de Modelos de Clasificación Eólica ---")
display(df_resultados_clf_eolica.sort_values(by='Accuracy', ascending=False))

--- Entrenando y evaluando modelos de CLASIFICACIÓN EÓLICA ---

--- Resultados para: Regresión Logística (Eólica) ---
Exactitud (Accuracy): 0.8750

Reporte de Clasificación:
              precision    recall  f1-score   support

        Alto       1.00      1.00      1.00         8
        Bajo       1.00      0.62      0.77         8
       Medio       0.73      1.00      0.84         8

    accuracy                           0.88        24
   macro avg       0.91      0.88      0.87        24
weighted avg       0.91      0.88      0.87        24

-----------------------------------
--- Resultados para: Random Forest Classifier (Eólica) ---
Exactitud (Accuracy): 0.8333

Reporte de Clasificación:
              precision    recall  f1-score   support

        Alto       1.00      1.00      1.00         8
        Bajo       0.83      0.62      0.71         8
       Medio       0.70      0.88      0.78         8

    accuracy                           0.83        24
   macro avg       0.8

,Accuracy
Gradient Boosting Classifier,0.958333
Regresión Logística,0.875000
Random Forest Classifier,0.833333


In [34]:
# Celda 9: División Train/Test para Energía Hídrica

# División para la tarea de Regresión Hídrica
X_train_hidrica_reg, X_test_hidrica_reg, y_train_hidrica_reg, y_test_hidrica_reg = train_test_split(
    X, y_regresion_hidrica, test_size=0.2, random_state=42
)

# División para la tarea de Clasificación Hídrica
X_train_hidrica_clf, X_test_hidrica_clf, y_train_hidrica_clf, y_test_hidrica_clf = train_test_split(
    X, y_clasificacion_hidrica, test_size=0.2, random_state=42, stratify=y_clasificacion_hidrica
)

print("Divisiones Train/Test completadas para los modelos HÍDRICOS.")

Divisiones Train/Test completadas para los modelos HÍDRICOS.


In [35]:
# Celda 10: Entrenamiento y Evaluación de Modelos de Regresión Hídrica

resultados_regresion_hidrica = {}

print("--- Entrenando y evaluando modelos de REGRESIÓN HÍDRICA ---\n")

for nombre, modelo in modelos_regresion.items():
    pipeline_reg = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('regressor', modelo)])
    pipeline_reg.fit(X_train_hidrica_reg, y_train_hidrica_reg)
    y_pred = pipeline_reg.predict(X_test_hidrica_reg)
    
    mae = mean_absolute_error(y_test_hidrica_reg, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_hidrica_reg, y_pred))
    r2 = r2_score(y_test_hidrica_reg, y_pred)
    
    resultados_regresion_hidrica[nombre] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
    
    print(f"--- Resultados para: {nombre} (Hídrica) ---")
    print(f"Error Absoluto Medio (MAE): {mae:.2f} mm")
    print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f} mm")
    print(f"Coeficiente de Determinación (R²): {r2:.4f}")
    print("-" * 35)

df_resultados_reg_hidrica = pd.DataFrame(resultados_regresion_hidrica).T
print("\n--- Tabla Comparativa de Modelos de Regresión Hídrica ---")
display(df_resultados_reg_hidrica.sort_values(by='RMSE'))

--- Entrenando y evaluando modelos de REGRESIÓN HÍDRICA ---

--- Resultados para: Regresión Lineal (Hídrica) ---
Error Absoluto Medio (MAE): 0.00 mm
Raíz del Error Cuadrático Medio (RMSE): 0.00 mm
Coeficiente de Determinación (R²): 1.0000
-----------------------------------
--- Resultados para: Random Forest Regressor (Hídrica) ---
Error Absoluto Medio (MAE): 45.35 mm
Raíz del Error Cuadrático Medio (RMSE): 132.69 mm
Coeficiente de Determinación (R²): 0.9699
-----------------------------------
--- Resultados para: Gradient Boosting Regressor (Hídrica) ---
Error Absoluto Medio (MAE): 34.05 mm
Raíz del Error Cuadrático Medio (RMSE): 108.09 mm
Coeficiente de Determinación (R²): 0.9800
-----------------------------------

--- Tabla Comparativa de Modelos de Regresión Hídrica ---


,MAE,RMSE,R2
Regresión Lineal,0.000262,0.000343,1.000000
Gradient Boosting Regressor,34.046157,108.094037,0.980016
Random Forest Regressor,45.351054,132.694880,0.969885


In [36]:
# Celda 11: Entrenamiento y Evaluación de Modelos de Clasificación Hídrica

# Codificar la variable objetivo 'viabilidad_hidrica' a números
le_hidrica = LabelEncoder()
y_train_hidrica_clf_encoded = le_hidrica.fit_transform(y_train_hidrica_clf)
y_test_hidrica_clf_encoded = le_hidrica.transform(y_test_hidrica_clf)

resultados_clasificacion_hidrica = {}

print("--- Entrenando y evaluando modelos de CLASIFICACIÓN HÍDRICA ---\n")

for nombre, modelo in modelos_clasificacion.items():
    pipeline_clf = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', modelo)])
    pipeline_clf.fit(X_train_hidrica_clf, y_train_hidrica_clf_encoded)
    y_pred_encoded = pipeline_clf.predict(X_test_hidrica_clf)
    
    accuracy = accuracy_score(y_test_hidrica_clf_encoded, y_pred_encoded)
    resultados_clasificacion_hidrica[nombre] = {'Accuracy': accuracy}
    
    print(f"--- Resultados para: {nombre} (Hídrica) ---")
    print(f"Exactitud (Accuracy): {accuracy:.4f}")
    print("\nReporte de Clasificación:")
    print(classification_report(y_test_hidrica_clf_encoded, y_pred_encoded, target_names=le_hidrica.classes_))
    print("-" * 35)

df_resultados_clf_hidrica = pd.DataFrame(resultados_clasificacion_hidrica).T
print("\n--- Tabla Comparativa de Modelos de Clasificación Hídrica ---")
display(df_resultados_clf_hidrica.sort_values(by='Accuracy', ascending=False))

--- Entrenando y evaluando modelos de CLASIFICACIÓN HÍDRICA ---

--- Resultados para: Regresión Logística (Hídrica) ---
Exactitud (Accuracy): 0.8750

Reporte de Clasificación:
              precision    recall  f1-score   support

        Alto       0.80      1.00      0.89         8
        Bajo       1.00      0.88      0.93         8
       Medio       0.86      0.75      0.80         8

    accuracy                           0.88        24
   macro avg       0.89      0.88      0.87        24
weighted avg       0.89      0.88      0.87        24

-----------------------------------
--- Resultados para: Random Forest Classifier (Hídrica) ---
Exactitud (Accuracy): 0.9583

Reporte de Clasificación:
              precision    recall  f1-score   support

        Alto       1.00      1.00      1.00         8
        Bajo       1.00      0.88      0.93         8
       Medio       0.89      1.00      0.94         8

    accuracy                           0.96        24
   macro avg       

,Accuracy
Gradient Boosting Classifier,1.000000
Random Forest Classifier,0.958333
Regresión Logística,0.875000


In [37]:
# Celda 12: Guardar los mejores modelos entrenados y los codificadores para el despliegue

# Crear el directorio de despliegue si no existe
if not os.path.exists('../deployment'):
    os.makedirs('../deployment')

# --- GUARDAR MODELOS DE REGRESIÓN (Random Forest suele ser el mejor) ---
modelo_reg_solar = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('regressor', RandomForestRegressor(random_state=42, n_estimators=100))])
modelo_reg_solar.fit(X, y_regresion_solar) # Entrenar con TODOS los datos
joblib.dump(modelo_reg_solar, '../deployment/modelo_regresion_solar.pkl')

modelo_reg_eolica = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('regressor', RandomForestRegressor(random_state=42, n_estimators=100))])
modelo_reg_eolica.fit(X, y_regresion_eolica)
joblib.dump(modelo_reg_eolica, '../deployment/modelo_regresion_eolica.pkl')

modelo_reg_hidrica = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('regressor', RandomForestRegressor(random_state=42, n_estimators=100))])
modelo_reg_hidrica.fit(X, y_regresion_hidrica)
joblib.dump(modelo_reg_hidrica, '../deployment/modelo_regresion_hidrica.pkl')

# --- GUARDAR MODELOS DE CLASIFICACIÓN (Random Forest suele ser el mejor) ---
modelo_clf_solar = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', RandomForestClassifier(random_state=42, n_estimators=100))])
modelo_clf_solar.fit(X, le_solar.transform(y_clasificacion_solar))
joblib.dump(modelo_clf_solar, '../deployment/modelo_clasificacion_solar.pkl')

modelo_clf_eolica = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('classifier', RandomForestClassifier(random_state=42, n_estimators=100))])
modelo_clf_eolica.fit(X, le_eolica.transform(y_clasificacion_eolica))
joblib.dump(modelo_clf_eolica, '../deployment/modelo_clasificacion_eolica.pkl')

modelo_clf_hidrica = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('classifier', RandomForestClassifier(random_state=42, n_estimators=100))])
modelo_clf_hidrica.fit(X, le_hidrica.transform(y_clasificacion_hidrica))
joblib.dump(modelo_clf_hidrica, '../deployment/modelo_clasificacion_hidrica.pkl')

# --- GUARDAR CODIFICADORES Y DATASET ---
joblib.dump(le_solar, '../deployment/label_encoder_solar.pkl')
joblib.dump(le_eolica, '../deployment/label_encoder_eolica.pkl')
joblib.dump(le_hidrica, '../deployment/label_encoder_hidrica.pkl')

df.to_csv('../deployment/dataset_para_app_completo.csv', index=False)

print("✅ Todos los modelos, codificadores y el dataset han sido guardados para la aplicación.")

✅ Todos los modelos, codificadores y el dataset han sido guardados para la aplicación.
